## Step1: Install Feast

Install Feast on your local machine. Consider running following commands in case you are running into dependency issues while installing Feast:
<div class="alert alert-block alert-warning">
    
```shell
!pip install --upgrade PyYAML
!python3 -m pip install --upgrade pip
!pip install --ignore-installed PyYAML
```
</div>



In [1]:
!pwd
# Your current working directory should be .../sfguide-getting-started-snowpark-python-feast

/Users/rsaxena/Documents/GitHub/sfguide-getting-started-snowpark-python-feast


In [2]:
%%sh
pip install 'feast[snowflake]' -U -q
pip install Pygments -q

In [3]:
#Check if Feast is properly installed on your machine
!pip show feast

Name: feast
Version: 0.28.0
Summary: Python SDK for Feast
Home-page: https://github.com/feast-dev/feast
Author: Feast
Author-email: 
License: Apache
Location: /Users/rsaxena/opt/anaconda3/envs/getting_started_snowpark_python/lib/python3.8/site-packages
Requires: bowler, click, colorama, dask, dill, fastapi, fastavro, google-api-core, googleapis-common-protos, grpcio, grpcio-reflection, Jinja2, jsonschema, mmh3, numpy, pandas, pandavro, proto-plus, protobuf, pyarrow, pydantic, pygments, PyYAML, SQLAlchemy, tabulate, tenacity, toml, tqdm, typeguard, uvicorn
Required-by: 


## Step2: Create Feature Repository
A feature repository is a directory that contains the configuration of the feature store and individual features. This configuration is written as code (Python/YAML) and it's highly recommended that teams track it centrally using git.

### 2.a Open Terminal window to run following commands
Open terminal and run following commands in the **terminal window** to setup feature repository
<div class="alert alert-block alert-warning">

1. CD to the current working directory .../sfguide-getting-started-snowpark-python-feast/
2. Run following command on the terminal 
```shell
feast init -t snowflake customer_repo
```
3. Fill in Snowflake account details as prompted:

```shell
Snowflake Deployment URL: ...
Snowflake User Name: ...
Snowflake Password: ...
Snowflake Role Name: ACCOUNTADMIN
Snowflake Warehouse Name: LAB_S_WH
Snowflake Database Name: FEAST_SF
Should I upload example data to Snowflake (overwriting "customer_repo_feast_driver_hourly_stats" table)? [Y/n]: n
```

Feast repository should be successfully created with this message: 
```shell
Creating a new Feast repository in .../sfguide-getting-started-snowpark-python-feast/customer-churn-prediction/customer_repo
```
</div>

In [4]:
!pwd
# your current directory should be .../sfguide-getting-started-snowpark-python-feast

/Users/rsaxena/Documents/GitHub/sfguide-getting-started-snowpark-python-feast


Next, we will remove the sample data and quickstart files created by feast init command to avoid any confusion. You may also chose to create a .feastignore file to exclude the sample .py file.

In [5]:
%%sh
rm -r customer_repo/feature_repo/data
rm customer_repo/feature_repo/driver_repo.py 
rm customer_repo/README.md
rm customer_repo/test_workflow.py 

### 2.b Copy cust_repo.py and Review feature_store.yaml file 
A pre-configured .py file is provided as part of this lab. All the required feature definitions and configurations are provided in this file. Next, we will copy the provided .py to its intended directory and review the .yaml file, created in the steps above, using the commands below

In [6]:
# Copy the feature repository file provided as part of this guide to feature_repo folder
%cp cust_repo.py customer_repo/feature_repo

### Review feature_store.yaml created during feast init process
Notice that Snowflake is used as the processing engine i.e. push down processing for creation of features, and Snowflake is used as both online as well as offline feature store.

In [7]:
# CD to feature_repo folder
%cd customer_repo/feature_repo

/Users/rsaxena/Documents/GitHub/sfguide-getting-started-snowpark-python-feast/customer_repo/feature_repo


In [ ]:
!pygmentize feature_store.yaml

### Review cust_repo.py
This is the **core configuration** file where we have defined feature views, sources and feature service for the feature store setup

In [10]:
!pygmentize cust_repo.py

from datetime import timedelta

import pandas as pd
import yaml

from feast import (
    Entity,
    FeatureService,
    FeatureView,
    Field,
    PushSource,
    RequestSource,
    SnowflakeSource,
)

from feast.types import Float64, Int64, Bool, String

# Define an entity for the customer. You can think of an entity as a primary key used to
# fetch features.
customer = Entity(name="customer", join_keys=["CUSTOMERID"])

# Defines a data source from which feature values can be retrieved. Sources are queried when building training
# datasets or materializing features into an online store.
project_name = yaml.safe_load(open("feature_store.yaml"))["project"]

cust_demographics_source = SnowflakeSource(
    # The Snowflake table where features can be found
    database=yaml.safe_load(open("feature_store.yaml"))["offline_store"]["database"],
    table=f"DEMOGRAPHICS",
    # The event timestamp is used for point-in-time joins and for ensuring only
    # features within the TTL are returned

## Step3: Deploy Feature Store

Open terminal and run following commands in the **terminal window** to setup feature repository. Deploy the feature store by running `apply` from within the `feature_repo/` folder
<div class="alert alert-block alert-warning">
    
1. CD to the **./customer_repo/feature_repo** folder
2. Run the following command in the terminal window    
```shell
feast apply
```
    
Once the feature store is successfully deployed, you should see following output in the command line:

```shell

Created entity customer

Created feature view cust_services

Created feature view cust_demographics

Created feature service customer_info

    
Created entity customer
Created feature view cust_services
Created feature view cust_demographics
Created feature service customer_info

Deploying materialization functions for customer_repo

Deploying infrastructure for cust_services
Deploying infrastructure for cust_demographics
```
</div>

Feast creates **registry.db** in .../customer_repo/feature_repo/ folder. This is the registry of all feature information stored by Feast for this feature store.


## Step4: Test the Offline Feature Store Deployment

In [11]:
import pandas as pd
from feast import FeatureStore
from datetime import datetime

In [12]:
# Initialize the entity dataframe for which the features are required to be extracted from the feature store

custid = ['7090-ZyCMx', '1364-wJXMS', '6564-sLgIC']
customers = pd.DataFrame(
    {
       "event_timestamp": datetime.now(),
       "CUSTOMERID": custid,
    }
)

print(customers)

             event_timestamp  CUSTOMERID
0 2023-01-10 13:49:34.050855  7090-ZyCMx
1 2023-01-10 13:49:34.050855  1364-wJXMS
2 2023-01-10 13:49:34.050855  6564-sLgIC


In [13]:
# Generate training dataset from Feast offline feature store

fs = FeatureStore(repo_path=".")  # Initialize the feature store

feature_service = fs.get_feature_service("customer_info")
fs.get_historical_features(features=feature_service, entity_df=customers).to_snowflake(table_name='historical_features')
# get_historical_features utilises Snowflake compute to create offline features
# to_snowflake() store the offline features in the table name specified in the argument
# you may switch to SnowSight to view the table and offline feature records created through Feast
# you may also check the push down processing in Activity > Query History

In [14]:
# Generate scoring dataset from Feast online feature store

fs.materialize_incremental(end_date=datetime.now())

online_features = fs.get_online_features(
    features=feature_service, entity_rows=[{"CUSTOMERID": '7090-ZyCMx'}, {"CUSTOMERID": '1364-wJXMS'}, {"CUSTOMERID": '6564-sLgIC'}],
).to_dict()

online_features

# Note that the output dataset is in the format expected by models deployed as APIs for online inference

Materializing 2 feature views to 2023-01-10 13:49:51+11:00 into the snowflake.online online store.

cust_services from 2013-01-22 02:49:51+11:00 to 2023-01-10 13:49:51+11:00:
Snowflake Query ID: 01a98c09-3201-3e20-0000-1af50036758a
cust_demographics from 2013-01-22 02:50:15+11:00 to 2023-01-11 00:49:51+11:00:
Snowflake Query ID: 01a98c0a-3201-3e79-0000-1af50036860a


{'CUSTOMERID': ['7090-ZyCMx', '1364-wJXMS', '6564-sLgIC'],
 'PARTNER': [False, False, False],
 'DEPENDENTS': [True, True, True],
 'SENIORCITIZEN': [False, False, True],
 'GENDER': ['Female', 'Female', 'Male'],
 'PAPERLESSBILLING': ['true', 'true', 'true'],
 'MULTIPLELINES': ['No', 'Yes', 'Yes'],
 'PAYMENTMETHOD': ['Electronic check', 'Electronic check', 'Electronic check'],
 'TECHSUPPORT': ['No', 'No', 'No'],
 'MONTHLYCHARGES': [70.7, 99.65, 95.45],
 'PHONESERVICE': ['Yes', 'Yes', 'Yes'],
 'ONLINEBACKUP': ['No', 'No', 'No'],
 'ONLINESECURITY': ['No', 'No', 'No'],
 'DEVICEPROTECTION': ['No', 'Yes', 'No'],
 'STREAMINGTV': ['No', 'Yes', 'Yes'],
 'STREAMINGMOVIES': ['No', 'Yes', 'Yes'],
 'TOTALCHARGES': [151.65, 820.5, 1752.55],
 'TENUREMONTHS': [2.0, 8.0, 18.0],
 'CHURNVALUE': [1.0, 1.0, 1.0],
 'INTERNETSERVICE': ['Fiber optic', 'Fiber optic', 'Fiber optic'],
 'CONTRACT': ['Month-to-month', 'Month-to-month', 'Month-to-month']}